# MDF data analysis

In [1]:
from load_data import train, test, BIG, TRAIN, TEST
import numpy as np
import pandas as pd

TRAIN (8564, 42)
TEST  (3671, 42)
BIG  (12235, 43)


In [ ]:
from visualization_helper import display_groups, display_unique_count

In [2]:
print train.shape, train.columns.values

(8564, 42) [u'libelle' u'libelle_plaquette' u'libelle_ampoule' u'libelle_flacon'
 u'libelle_tube' u'libelle_stylo' u'libelle_seringue' u'libelle_pilulier'
 u'libelle_sachet' u'libelle_comprime' u'libelle_gelule' u'libelle_film'
 u'libelle_poche' u'libelle_capsule' u'nb_plaquette' u'nb_ampoule'
 u'nb_flacon' u'nb_tube' u'nb_stylo' u'nb_seringue' u'nb_pilulier'
 u'nb_sachet' u'nb_comprime' u'nb_gelule' u'nb_film' u'nb_poche'
 u'nb_capsule' u'nb_ml' u'statut' u'etat commerc' u'agrement col'
 u'tx rembours' u'forme pharma' u'voies admin' u'statut admin'
 u'date declar annee' u'date amm annee' u'type proc' u'titulaires'
 u'substances' u'prix' 'source']


Remove some useless columns

In [3]:
train__noid_noprix = train.drop([u'prix',u'libelle'], axis=1)
print train__noid_noprix.shape, train__noid_noprix.columns.values

(8564, 40) [u'libelle_plaquette' u'libelle_ampoule' u'libelle_flacon' u'libelle_tube'
 u'libelle_stylo' u'libelle_seringue' u'libelle_pilulier' u'libelle_sachet'
 u'libelle_comprime' u'libelle_gelule' u'libelle_film' u'libelle_poche'
 u'libelle_capsule' u'nb_plaquette' u'nb_ampoule' u'nb_flacon' u'nb_tube'
 u'nb_stylo' u'nb_seringue' u'nb_pilulier' u'nb_sachet' u'nb_comprime'
 u'nb_gelule' u'nb_film' u'nb_poche' u'nb_capsule' u'nb_ml' u'statut'
 u'etat commerc' u'agrement col' u'tx rembours' u'forme pharma'
 u'voies admin' u'statut admin' u'date declar annee' u'date amm annee'
 u'type proc' u'titulaires' u'substances' 'source']


Check for dupicate rows.

In [4]:
duplicated = train__noid_noprix.duplicated()
np.sum(duplicated)

1164

In [5]:
train__noid_noprix_unique = train__noid_noprix.drop_duplicates()
len(train__noid_noprix_unique)

7400

Lets look at the columns with only one unique value.

In [6]:
def drop_const_cols(df):
    return df.loc[:, (df != df.ix[0]).any()]

train__noid_noprix_unique_noconst = drop_const_cols(train__noid_noprix_unique)
print train__noid_noprix_unique.shape, '->', train__noid_noprix_unique_noconst.shape


(7400, 40) -> (7400, 39)


In [7]:
print "Dropped const columns : "
if len(train__noid_noprix_unique.columns) > len(train__noid_noprix_unique_noconst.columns):
    print train__noid_noprix_unique.columns.difference(train__noid_noprix_unique_noconst.columns).values


Dropped const columns : 
['source']


Columns description : 

In [11]:
type_groups_func = lambda df : df.columns.to_series().groupby(df.dtypes)
type_groups = type_groups_func(train__noid_noprix_unique_noconst)

In [12]:
for key, value in type_groups.groups.items():
    print '\n--', key, '--'
    print "\n\t", "\n\t".join(value.values)


-- object --

	statut
	etat commerc
	agrement col
	tx rembours
	forme pharma
	voies admin
	statut admin
	type proc
	titulaires
	substances

-- int64 --

	libelle_plaquette
	libelle_ampoule
	libelle_flacon
	libelle_tube
	libelle_stylo
	libelle_seringue
	libelle_pilulier
	libelle_sachet
	libelle_comprime
	libelle_gelule
	libelle_film
	libelle_poche
	libelle_capsule
	nb_plaquette
	nb_ampoule
	nb_flacon
	nb_tube
	nb_stylo
	nb_seringue
	nb_pilulier
	nb_sachet
	nb_comprime
	nb_gelule
	nb_film
	nb_poche
	nb_capsule
	date declar annee
	date amm annee

-- float64 --

	nb_ml


Identify and separate the numeric and non numeric rows.
Report NaN values on numerical columns

In [15]:
numeric_cols = []
nonnum_cols = []
for key in type_groups.groups:
    if np.issubdtype(key, np.number):
        numeric_cols.extend(type_groups.get_group(key).values)
    else:
        nonnum_cols.extend(type_groups.get_group(key).values)
        
nb_isnan = np.sum(train__noid_noprix_unique_noconst[numeric_cols].apply(np.isnan))

In [16]:
print nb_isnan

libelle_plaquette    0
libelle_ampoule      0
libelle_flacon       0
libelle_tube         0
libelle_stylo        0
libelle_seringue     0
libelle_pilulier     0
libelle_sachet       0
libelle_comprime     0
libelle_gelule       0
libelle_film         0
libelle_poche        0
libelle_capsule      0
nb_plaquette         0
nb_ampoule           0
nb_flacon            0
nb_tube              0
nb_stylo             0
nb_seringue          0
nb_pilulier          0
nb_sachet            0
nb_comprime          0
nb_gelule            0
nb_film              0
nb_poche             0
nb_capsule           0
date declar annee    0
date amm annee       0
nb_ml                0
dtype: int64


Identify non numeric rows.

In [17]:
train__noid_noprix_unique_noconst[nonnum_cols].tail(5)

,statut,etat commerc,agrement col,tx rembours,forme pharma,voies admin,statut admin,type proc,titulaires,substances
8553,Présentation active,Déclaration de commercialisation,oui,65%,solution injectable,intraveineuse,Autorisation active,Procédure nationale,BRACCO IMAGING FRANCE,GADOTÉRIDOL
8556,Présentation active,Déclaration de commercialisation,oui,65%,solution injectable,sous-cutanée,Autorisation active,Procédure centralisée,BRISTOL MYERS SQUIBB PHARMA (GRANDE BRETAGNE),ABATACEPT
8557,Présentation active,Déclaration de commercialisation,oui,65%,gélule,orale,Autorisation active,Procédure nationale,BIOGARAN,"FLUVASTATINE, FLUVASTATINE SODIQUE"
8559,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,comprimé orodispersible,orale,Autorisation active,Procédure décentralisée,RATIOPHARM (ALLEMAGNE),OLANZAPINE
8560,Présentation active,Déclaration de commercialisation,oui,65%,solution injectable,sous-cutanée,Autorisation active,Procédure centralisée,MERCK SERONO EUROPE (ROYAUME-UNI),INTERFÉRON BÊTA-1A


In [27]:

for col_name in nonnum_cols:
    l = train__noid_noprix_unique_noconst[col_name].unique().tolist()
    print '\n--', col_name, '--'
    print "\n\t", "\n\t".join(l)


-- statut --

	Présentation active
	Présentation abrogée

-- etat commerc --

	Déclaration de commercialisation
	Déclaration d'arrêt de commercialisation
	Arrêt de commercialisation (le médicament n'a plus d'autorisation)
	Déclaration de suspension de commercialisation

-- agrement col --

	oui
	non

-- tx rembours --

	65%
	100%
	15%
	30%
	0%

-- forme pharma --

	comprimé pelliculé
	comprimé à croquer
	gélule à libération prolongée
	granulés à libération prolongée
	comprimé
	capsule molle
	comprimé sécable
	dispositif
	poudre pour inhalation
	solution buvable gouttes
	comprimé dispersible orodispersible
	comprimé pelliculé sécable
	comprimé quadrisécable
	comprimé pelliculé à libération prolongée
	gélule gastro-résistant(e)
	gélule
	collyre en solution
	poudre et  solvant pour solution injectable
	solution gouttes
	comprimé orodispersible
	solution injectable
	émulsion fluide pour application
	comprimé effervescent(e) sécable
	suppositoire
	poudre pour solution pour irrigation vésic